# Script Goals:

1. Join Lease Records from database to GIS shapefile for exporting to Tableau
    
    
## Script Overview
1.  Pull lease records from database joined with us official counties, state, fips data 
2.  Quality control GIS shapefile join

In [23]:
import pandas as pd
import geopandas as gp
import datetime as dt
import pdb
import numpy as np
import requests as req

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Read in Source Data

### Connect to Company Database to Load Lease Data

In [53]:
import pymysql
from db_config import mysql_config

In [54]:
db = pymysql.connect(host = mysql_config['host'], user = mysql_config['user'], password = mysql_config['password'], database = mysql_config['database'])

In [26]:
#pull in details table with official county names
sql = '''
SELECT d.*, o.official_county, o.fips FROM details d
INNER JOIN official_counties o
ON o.lease_no = d.lease_no;
'''

leases = pd.read_sql(sql, db)
leases.info()
leases.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2175 entries, 0 to 2174
Data columns (total 18 columns):
LEASE_NO                   2175 non-null object
PLOTTED                    2175 non-null object
GRANTEE                    2175 non-null object
SALE_DATE                  2157 non-null object
EFFECTIVE_DATE_OF_LEASE    2081 non-null object
EXPIRATION_DATE            2081 non-null object
ACRES                      2108 non-null float64
COUNTY                     2128 non-null object
ST                         2133 non-null object
STATUS_COMMENTS            908 non-null object
EXPIRATION_YEAR            2076 non-null float64
TOWNSHIP                   1332 non-null object
RANGE_PLSS                 1331 non-null object
DESCRIPTION_I              1930 non-null object
DESCRIPTION_II             1868 non-null object
DESCRIPTION_III            844 non-null object
official_county            2175 non-null object
fips                       2175 non-null int64
dtypes: float64(2), int64(1), 

,LEASE_NO,PLOTTED,GRANTEE,SALE_DATE,EFFECTIVE_DATE_OF_LEASE,EXPIRATION_DATE,ACRES,COUNTY,ST,STATUS_COMMENTS,EXPIRATION_YEAR,TOWNSHIP,RANGE_PLSS,DESCRIPTION_I,DESCRIPTION_II,DESCRIPTION_III,official_county,fips
0,ALES51293,NO,R&R Royalty,2002-03-28,2002-05-01,2012-05-01,80.25,LAMAR,AL,Expired 2012,2012.0,None,None,SEE STIPULATIONS & LSE NOTICE,"T16S,R16W HUNTSVILLE MERIDIAN","SEC 5, N2NE","Lamar, AL",1075
1,ALES51294,NO,R&R Royalty,2002-03-28,2002-05-01,2012-05-01,80.70,LAMAR,AL,Expired 2012,2012.0,None,None,SEE STIPULATIONS & LSE NOTICE,"T16S,R16W HUNTSVILLE MERIDIAN","SEC 11, NESW; SEC 20, NWNE","Lamar, AL",1075
2,ALES55113,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,161.55,COVINGTON,AL,EXP. AGREEMENT - PRUET 3D/CRAIN- EXPIRED,2018.0,None,None,CONECUH NATL FOREST,"T1N,R15E,ST.STEPHENS MERIDAN","SEC 5, N2NW; SEC 6, SESW; SEC 8, NWSW","Covington, AL",1039


# Joining Manipulated Master leases with GIS Shapefile for Tableau

In [27]:
shapefile = gp.read_file("zip://Source Files/Master GIS Shapefile/BLM Master 8-2021.zip")
shapefile

,lot_no,LEASE_NO,EFFECTIVE,SALE_DATE,Exp_date,COUNTY,ST,Ownership,GeneralLoc,Due_Year,Acres,COMMENTS,Est_Bonus,Comments_1,Shape_Leng,Shape_Area,ObjectID,geometry
0,72266,WYW 190484,None,2020-12-15,None,Niobrara,WY,R&R Royalty,None,0.0,840.000,Lease yet to be issued,0,Lease yet to be issued,11018.376216,6.377650e+06,1793,"POLYGON Z ((-11639970.496 5340973.580 0.000, -..."
1,72267,WYW 190485,None,2020-12-15,None,Niobrara,WY,R&R Royalty,None,0.0,1280.000,Lease yet to be issued,0,Lease yet to be issued,13275.331252,9.790726e+06,1792,"POLYGON Z ((-11635543.812 5338747.382 0.000, -..."
2,72268,WYW 190486,None,2020-12-15,None,Niobrara,WY,R&R Royalty,None,0.0,960.000,Lease yet to be issued,0,Lease yet to be issued,11051.594313,7.316665e+06,1791,"POLYGON Z ((-11633336.189 5338743.869 0.000, -..."
3,72262,WYW 190480,None,2020-12-15,None,Niobrara,WY,R&R Royalty,None,0.0,641.810,Lease yet to be issued,0,Lease yet to be issued,8834.223784,4.875540e+06,1797,"POLYGON Z ((-11637213.500 5343726.602 0.000, -..."
4,72312,WYW 190529,None,2020-12-15,None,Converse,WY,R&R Royalty,None,0.0,307.800,Lease yet to be issued,0,Lease yet to be issued,6584.338297,2.371568e+06,1782,"POLYGON Z ((-11723601.654 5349650.980 0.000, -..."
5,72346,WYW 190563,None,2020-12-15,None,Converse,WY,R&R Royalty,None,0.0,160.000,Lease yet to be issued,0,Lease yet to be issued,4436.075762,1.229848e+06,1777,"POLYGON Z ((-11726899.534 5349637.178 0.000, -..."
6,72339,WYW 190556,None,2020-12-15,None,Converse,WY,R&R Royalty,None,0.0,160.000,Lease yet to be issued,0,Lease yet to be issued,4472.385354,1.250105e+06,1779,"POLYGON Z ((-11727486.350 5353533.064 0.000, -..."
7,72495,WYW 190711,None,2020-12-17,None,Sublette,WY,R&R Royalty,None,0.0,640.000,Lease yet to be issued,0,Lease yet to be issued,8726.389708,4.750538e+06,1770,"POLYGON Z ((-12224649.873 5248083.475 0.000, -..."
8,72294,WYW 190512,None,2020-12-15,None,Converse,WY,R&R Royalty,None,0.0,40.000,Lease yet to be issued,0,Lease yet to be issued,2218.866703,3.076629e+05,1789,"POLYGON Z ((-11683378.069 5338603.633 0.000, -..."
9,72301,WYW 190518,None,2020-12-15,None,Converse,WY,R&R Royalty,None,0.0,160.650,Lease yet to be issued,0,Lease yet to be issued,4499.943551,1.264170e+06,1787,"POLYGON Z ((-11707422.084 5357974.226 0.000, -..."


## Inner Join for Quality Control

In [28]:
shapeTableau = pd.merge(left = leases, right = shapefile[["LEASE_NO", "geometry"]] , on = "LEASE_NO")


### Checking if all leases in GIS Shapefile Match up

In [29]:
print('total leases in leases df from db: {}'.format(leases.count()['LEASE_NO']))
print('total leases in shapefile: {}'.format(shapefile.count()['LEASE_NO']))
print('total leases in Joined Tableau shapefile: {}'.format(shapeTableau.count()['LEASE_NO']))
print('difference is {}'.format(shapefile.count()['LEASE_NO'] - shapeTableau.count()['LEASE_NO']))

total leases in leases df from db: 2175
total leases in shapefile: 1874
total leases in Joined Tableau shapefile: 1874
difference is 0


## Troubleshooting Leases in Shapefile that Are not matching in Database (if applicable)

In [30]:
shapefile[~shapefile['LEASE_NO'].isin(leases['LEASE_NO'])]

shapefile[~shapefile['LEASE_NO'].isin(leases['LEASE_NO'])]['LEASE_NO']

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

Empty GeoDataFrame
Columns: [lot_no, LEASE_NO, EFFECTIVE, SALE_DATE, Exp_date, COUNTY, ST, Ownership, GeneralLoc, Due_Year, Acres, COMMENTS, Est_Bonus, Comments_1, Shape_Leng, Shape_Area, ObjectID, geometry]
Index: []

Series([], Name: LEASE_NO, dtype: object)

In [31]:
shapefile[~shapefile['LEASE_NO'].isin(leases['LEASE_NO'].apply(lambda x: x.strip()))]
shapefile[~shapefile['LEASE_NO'].apply(lambda x: x.strip()).isin(leases['LEASE_NO'])]

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

Empty GeoDataFrame
Columns: [lot_no, LEASE_NO, EFFECTIVE, SALE_DATE, Exp_date, COUNTY, ST, Ownership, GeneralLoc, Due_Year, Acres, COMMENTS, Est_Bonus, Comments_1, Shape_Leng, Shape_Area, ObjectID, geometry]
Index: []

IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

Empty GeoDataFrame
Columns: [lot_no, LEASE_NO, EFFECTIVE, SALE_DATE, Exp_date, COUNTY, ST, Ownership, GeneralLoc, Due_Year, Acres, COMMENTS, Est_Bonus, Comments_1, Shape_Leng, Shape_Area, ObjectID, geometry]
Index: []

In [32]:
pd.read_sql('''select * from details 
where sale_date = '2020-12-17';''', db)

,LEASE_NO,PLOTTED,GRANTEE,SALE_DATE,EFFECTIVE_DATE_OF_LEASE,EXPIRATION_DATE,ACRES,COUNTY,ST,STATUS_COMMENTS,EXPIRATION_YEAR,TOWNSHIP,RANGE_PLSS,DESCRIPTION_I,DESCRIPTION_II,DESCRIPTION_III
0,ARES 059723,YES,R&R Royalty,2020-12-17,2021-06-01,2031-06-01,120.000,Faulkner,AR,None,NaN,7N,12W,5TH PM,"SEC 14 NE 1/4 NW 1/4, S 1/2 NW 1/4",None
1,MSES 059726,YES,R&R Royalty,2020-12-17,2021-06-01,2031-06-01,81.640,Stone,MS,None,2031.0,2S,9W,ST. STEPHENS MER,"SEC 31 E 1/2 W 1/2 NE 1/4, NW 1/4 SE 1/4",None
2,MSES 059734,YES,R&R Royalty,2020-12-17,2021-06-01,2031-06-01,122.630,Jones,MS,None,2031.0,7N,14,ST. STEPHENS MER,\nSec. 36 NESW that part lying west of Monroe ...,"Sec. 36 NW 1/4 SW 1/4, S 1/2 SW 1/4 NW 1/4, NE..."
3,MSES 059735,YES,R&R Royalty,2020-12-17,2021-06-01,2031-06-01,79.145,Franklin,MS,None,2031.0,5N,3E,WASHINGTON MER,\nSec. 37 S 1/2 SW 1/4,None
4,WYW 190710,YES,R&R Royalty,2020-12-17,None,None,1280.000,Sublette,WY,LEASE NOT ISSUED,NaN,30N,109W,"T. 30 N., R. 109 W., 6TH PM",Sec. 9 ALL;Sec. 21 N2;Sec. 22 N2,None
5,WYW 190711,YES,R&R Royalty,2020-12-17,None,None,640.000,Sublette,WY,LEASE NOT ISSUED,NaN,30N,109W,"T. 30 N., R. 109 W., 6TH PM",Sec. 15 ALL,None


In [33]:
failedGISLeases = shapefile[~shapefile['LEASE_NO'].isin(leases['LEASE_NO'])]['LEASE_NO']
failedGISLeases

Series([], Name: LEASE_NO, dtype: object)

In [34]:
from fuzzywuzzy import process

for gislease in failedGISLeases:
    print(gislease)
    process.extract(gislease, leases['LEASE_NO'])


## Left Join to Retain Leases in Database that were/are not plotted

In [35]:
shapeTableau = pd.merge(left = leases, right = shapefile[['LEASE_NO', 'geometry']], on = 'LEASE_NO', how = 'left')

shapeTableau
shapeTableau.info()

,LEASE_NO,PLOTTED,GRANTEE,SALE_DATE,EFFECTIVE_DATE_OF_LEASE,EXPIRATION_DATE,ACRES,COUNTY,ST,STATUS_COMMENTS,EXPIRATION_YEAR,TOWNSHIP,RANGE_PLSS,DESCRIPTION_I,DESCRIPTION_II,DESCRIPTION_III,official_county,fips,geometry
0,ALES51293,NO,R&R Royalty,2002-03-28,2002-05-01,2012-05-01,80.25,LAMAR,AL,Expired 2012,2012.0,None,None,SEE STIPULATIONS & LSE NOTICE,"T16S,R16W HUNTSVILLE MERIDIAN","SEC 5, N2NE","Lamar, AL",1075,None
1,ALES51294,NO,R&R Royalty,2002-03-28,2002-05-01,2012-05-01,80.70,LAMAR,AL,Expired 2012,2012.0,None,None,SEE STIPULATIONS & LSE NOTICE,"T16S,R16W HUNTSVILLE MERIDIAN","SEC 11, NESW; SEC 20, NWNE","Lamar, AL",1075,None
2,ALES55113,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,161.55,COVINGTON,AL,EXP. AGREEMENT - PRUET 3D/CRAIN- EXPIRED,2018.0,None,None,CONECUH NATL FOREST,"T1N,R15E,ST.STEPHENS MERIDAN","SEC 5, N2NW; SEC 6, SESW; SEC 8, NWSW","Covington, AL",1039,MULTIPOLYGON Z (((-9637686.462 3640737.307 0.0...
3,ALES55116,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1470.17,COVINGTON,AL,None,2018.0,None,None,CONECUH NATL FOREST,"T1N,R15E,ST.STEPHENS MERIDAN","ACREAGE IN SEC 15, 16, 18, 21 & 22; SEE LSE FO...","Covington, AL",1039,MULTIPOLYGON Z (((-9638158.080 3639784.474 0.0...
4,ALES55117,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,479.55,COVINGTON,AL,None,2018.0,None,None,CONECUH NATL FOREST,"T1N,R15E,ST.STEPHENS MERIDAN","SEC 27,SENW,SW; SEC 28,E2NE; SEC 33, NENE; SEC...","Covington, AL",1039,"POLYGON Z ((-9634322.330 3635634.953 0.000, -9..."
5,ALES55118,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,685.17,ESCAMBIA,AL,LEASE AMENDED. 320 ACR. CANC. SEC.11 REMOVED. ...,2018.0,None,None,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 8,N2NW,E2SE; SEC 9,SENE, S2; SEC15,N2,SW,W2SE","Escambia, AL",1053,MULTIPOLYGON Z (((-9655141.016 3651477.625 0.0...
6,ALES55119,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1720.42,ESCAMBIA,AL,None,2018.0,None,None,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 16,ALL; SEC 19,E2SW; SEC20,W2NE,SENE,NW,SE...","Escambia, AL",1053,MULTIPOLYGON Z (((-9661792.654 3647735.086 0.0...
7,ALES55121,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1480.33,ESCAMBIA,AL,EXP. AGREEMENT - PRUET 3D/CRAIN- EXPIRED,2018.0,None,None,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 27,ALL; SEC 28,ALL; SEC29 SENW,SW","Escambia, AL",1053,MULTIPOLYGON Z (((-9659903.478 3645833.341 0.0...
8,ALES55122,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1860.00,ESCAMBIA,AL,None,2018.0,None,None,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 30,E2SE, E2E2W2SE; SEC32,E2; SEC33,ALL; SE...","Escambia, AL",1053,MULTIPOLYGON Z (((-9658958.067 3645838.302 0.0...
9,ALES55123,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,2316.75,COVINGTON,AL,None,2018.0,None,None,CONECUH NATL FOREST,"T2N,R15E,ST.STEPHENS MERIDIAN","SEC 2,SWN2,SWNW,S2; SEC 3,ALL; SEC4,ALL; SEC 5...","Covington, AL",1039,"POLYGON Z ((-9632636.472 3655503.011 0.000, -9..."


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2174
Data columns (total 19 columns):
LEASE_NO                   2175 non-null object
PLOTTED                    2175 non-null object
GRANTEE                    2175 non-null object
SALE_DATE                  2157 non-null object
EFFECTIVE_DATE_OF_LEASE    2081 non-null object
EXPIRATION_DATE            2081 non-null object
ACRES                      2108 non-null float64
COUNTY                     2128 non-null object
ST                         2133 non-null object
STATUS_COMMENTS            908 non-null object
EXPIRATION_YEAR            2076 non-null float64
TOWNSHIP                   1332 non-null object
RANGE_PLSS                 1331 non-null object
DESCRIPTION_I              1930 non-null object
DESCRIPTION_II             1868 non-null object
DESCRIPTION_III            844 non-null object
official_county            2175 non-null object
fips                       2175 non-null int64
geometry                   187

# Convert Datetime to String for Export

In [36]:
#converting shapefile columns to strings from datetime for esri compatibility

shapeTableau.info()

for col in list(filter(lambda x: "DATE" in x.upper(), shapeTableau.columns)):
    shapeTableau[col] = shapeTableau[col].astype(str)

shapeTableau.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2175 entries, 0 to 2174
Data columns (total 19 columns):
LEASE_NO                   2175 non-null object
PLOTTED                    2175 non-null object
GRANTEE                    2175 non-null object
SALE_DATE                  2157 non-null object
EFFECTIVE_DATE_OF_LEASE    2081 non-null object
EXPIRATION_DATE            2081 non-null object
ACRES                      2108 non-null float64
COUNTY                     2128 non-null object
ST                         2133 non-null object
STATUS_COMMENTS            908 non-null object
EXPIRATION_YEAR            2076 non-null float64
TOWNSHIP                   1332 non-null object
RANGE_PLSS                 1331 non-null object
DESCRIPTION_I              1930 non-null object
DESCRIPTION_II             1868 non-null object
DESCRIPTION_III            844 non-null object
official_county            2175 non-null object
fips                       2175 non-null int64
geometry                   187

In [39]:
shapeTableau = gp.GeoDataFrame(shapeTableau)

In [16]:
shapeTableau.to_file("Output Lists/Tableau Final Shape/Public Website Version/PublicTableauShape.shp")

# Internal Version with Investment Info

In [70]:
# fullMaster = pd.read_excel("Source Files/Internal Master leases/Updated R&R & Magnum Master - BLM Leases.xlsx",
#                             header = 9)

investmentSql = '''
Select d.*, c.OFFICIAL_COUNTY, c.FIPS, i.YEARS_RENEWED, i.RENTALS_PAID, i.TOTAL_INVESTMENT, i.Estimated_Bonus from details d
join official_counties c
on c.lease_no = d.lease_no
join investment i
on i.lease_no = c.lease_no;
'''

internalLeases = pd.read_sql(investmentSql, db)
internalLeases

,LEASE_NO,PLOTTED,GRANTEE,SALE_DATE,EFFECTIVE_DATE_OF_LEASE,EXPIRATION_DATE,ACRES,COUNTY,ST,STATUS_COMMENTS,...,RANGE_PLSS,DESCRIPTION_I,DESCRIPTION_II,DESCRIPTION_III,OFFICIAL_COUNTY,FIPS,YEARS_RENEWED,RENTALS_PAID,TOTAL_INVESTMENT,Estimated_Bonus
0,ALES51293,NO,R&R Royalty,2002-03-28,2002-05-01,2012-05-01,80.25,LAMAR,AL,Expired 2012,...,None,SEE STIPULATIONS & LSE NOTICE,"T16S,R16W HUNTSVILLE MERIDIAN","SEC 5, N2NE","Lamar, AL",1075,10.0,1970.0,2302.5,12.58130
1,ALES51294,NO,R&R Royalty,2002-03-28,2002-05-01,2012-05-01,80.70,LAMAR,AL,Expired 2012,...,None,SEE STIPULATIONS & LSE NOTICE,"T16S,R16W HUNTSVILLE MERIDIAN","SEC 11, NESW; SEC 20, NWNE","Lamar, AL",1075,10.0,1970.0,3922.5,32.42590
2,ALES55113,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,161.55,COVINGTON,AL,EXP. AGREEMENT - PRUET 3D/CRAIN- EXPIRED,...,None,CONECUH NATL FOREST,"T1N,R15E,ST.STEPHENS MERIDAN","SEC 5, N2NW; SEC 6, SESW; SEC 8, NWSW","Covington, AL",1039,10.0,1970.0,3299.0,4.36420
3,ALES55116,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1470.17,COVINGTON,AL,None,...,None,CONECUH NATL FOREST,"T1N,R15E,ST.STEPHENS MERIDAN","ACREAGE IN SEC 15, 16, 18, 21 & 22; SEE LSE FO...","Covington, AL",1039,10.0,1970.0,28824.5,3.59762
4,ALES55117,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,479.55,COVINGTON,AL,None,...,None,CONECUH NATL FOREST,"T1N,R15E,ST.STEPHENS MERIDAN","SEC 27,SENW,SW; SEC 28,E2NE; SEC 33, NENE; SEC...","Covington, AL",1039,10.0,1970.0,9500.0,3.79167
5,ALES55118,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,685.17,ESCAMBIA,AL,LEASE AMENDED. 320 ACR. CANC. SEC.11 REMOVED. ...,...,None,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 8,N2NW,E2SE; SEC 9,SENE, S2; SEC15,N2,SW,W2SE","Escambia, AL",1053,10.0,1970.0,14637.0,5.34453
6,ALES55119,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1720.42,ESCAMBIA,AL,None,...,None,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 16,ALL; SEC 19,E2SW; SEC20,W2NE,SENE,NW,SE...","Escambia, AL",1053,10.0,1970.0,33699.5,3.58343
7,ALES55121,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1480.33,ESCAMBIA,AL,EXP. AGREEMENT - PRUET 3D/CRAIN- EXPIRED,...,None,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 27,ALL; SEC 28,ALL; SEC29 SENW,SW","Escambia, AL",1053,10.0,1970.0,29019.5,3.59696
8,ALES55122,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1860.00,ESCAMBIA,AL,None,...,None,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 30,E2SE, E2E2W2SE; SEC32,E2; SEC33,ALL; SE...","Escambia, AL",1053,10.0,1970.0,36410.0,3.57527
9,ALES55123,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,2316.75,COVINGTON,AL,None,...,None,CONECUH NATL FOREST,"T2N,R15E,ST.STEPHENS MERIDIAN","SEC 2,SWN2,SWNW,S2; SEC 3,ALL; SEC4,ALL; SEC 5...","Covington, AL",1039,10.0,1970.0,45321.5,3.56042


In [71]:
internalShape = pd.merge(internalLeases, shapefile[['LEASE_NO', 'geometry']], how = "left", on = "LEASE_NO")

internalShape

,LEASE_NO,PLOTTED,GRANTEE,SALE_DATE,EFFECTIVE_DATE_OF_LEASE,EXPIRATION_DATE,ACRES,COUNTY,ST,STATUS_COMMENTS,...,DESCRIPTION_I,DESCRIPTION_II,DESCRIPTION_III,OFFICIAL_COUNTY,FIPS,YEARS_RENEWED,RENTALS_PAID,TOTAL_INVESTMENT,Estimated_Bonus,geometry
0,ALES51293,NO,R&R Royalty,2002-03-28,2002-05-01,2012-05-01,80.25,LAMAR,AL,Expired 2012,...,SEE STIPULATIONS & LSE NOTICE,"T16S,R16W HUNTSVILLE MERIDIAN","SEC 5, N2NE","Lamar, AL",1075,10.0,1970.0,2302.5,12.58130,None
1,ALES51294,NO,R&R Royalty,2002-03-28,2002-05-01,2012-05-01,80.70,LAMAR,AL,Expired 2012,...,SEE STIPULATIONS & LSE NOTICE,"T16S,R16W HUNTSVILLE MERIDIAN","SEC 11, NESW; SEC 20, NWNE","Lamar, AL",1075,10.0,1970.0,3922.5,32.42590,None
2,ALES55113,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,161.55,COVINGTON,AL,EXP. AGREEMENT - PRUET 3D/CRAIN- EXPIRED,...,CONECUH NATL FOREST,"T1N,R15E,ST.STEPHENS MERIDAN","SEC 5, N2NW; SEC 6, SESW; SEC 8, NWSW","Covington, AL",1039,10.0,1970.0,3299.0,4.36420,MULTIPOLYGON Z (((-9637686.462 3640737.307 0.0...
3,ALES55116,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1470.17,COVINGTON,AL,None,...,CONECUH NATL FOREST,"T1N,R15E,ST.STEPHENS MERIDAN","ACREAGE IN SEC 15, 16, 18, 21 & 22; SEE LSE FO...","Covington, AL",1039,10.0,1970.0,28824.5,3.59762,MULTIPOLYGON Z (((-9638158.080 3639784.474 0.0...
4,ALES55117,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,479.55,COVINGTON,AL,None,...,CONECUH NATL FOREST,"T1N,R15E,ST.STEPHENS MERIDAN","SEC 27,SENW,SW; SEC 28,E2NE; SEC 33, NENE; SEC...","Covington, AL",1039,10.0,1970.0,9500.0,3.79167,"POLYGON Z ((-9634322.330 3635634.953 0.000, -9..."
5,ALES55118,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,685.17,ESCAMBIA,AL,LEASE AMENDED. 320 ACR. CANC. SEC.11 REMOVED. ...,...,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 8,N2NW,E2SE; SEC 9,SENE, S2; SEC15,N2,SW,W2SE","Escambia, AL",1053,10.0,1970.0,14637.0,5.34453,MULTIPOLYGON Z (((-9655141.016 3651477.625 0.0...
6,ALES55119,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1720.42,ESCAMBIA,AL,None,...,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 16,ALL; SEC 19,E2SW; SEC20,W2NE,SENE,NW,SE...","Escambia, AL",1053,10.0,1970.0,33699.5,3.58343,MULTIPOLYGON Z (((-9661792.654 3647735.086 0.0...
7,ALES55121,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1480.33,ESCAMBIA,AL,EXP. AGREEMENT - PRUET 3D/CRAIN- EXPIRED,...,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 27,ALL; SEC 28,ALL; SEC29 SENW,SW","Escambia, AL",1053,10.0,1970.0,29019.5,3.59696,MULTIPOLYGON Z (((-9659903.478 3645833.341 0.0...
8,ALES55122,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,1860.00,ESCAMBIA,AL,None,...,CONECUH NATL FOREST,"T2N,R13E,ST.STEPHENS MERIDAN","SEC 30,E2SE, E2E2W2SE; SEC32,E2; SEC33,ALL; SE...","Escambia, AL",1053,10.0,1970.0,36410.0,3.57527,MULTIPOLYGON Z (((-9658958.067 3645838.302 0.0...
9,ALES55123,YES,R&R Royalty,2007-12-20,2008-02-01,2018-02-01,2316.75,COVINGTON,AL,None,...,CONECUH NATL FOREST,"T2N,R15E,ST.STEPHENS MERIDIAN","SEC 2,SWN2,SWNW,S2; SEC 3,ALL; SEC4,ALL; SEC 5...","Covington, AL",1039,10.0,1970.0,45321.5,3.56042,"POLYGON Z ((-9632636.472 3655503.011 0.000, -9..."


In [74]:
internalShape = gp.GeoDataFrame(internalShape)

#converting shapefile columns to strings from datetime for esri compatibility
for col in list(filter(lambda x: "DATE" in x.upper(), internalShape.columns)):
    internalShape[col] = internalShape[col].astype(str)

internalShape.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 2175 entries, 0 to 2174
Data columns (total 23 columns):
LEASE_NO                   2175 non-null object
PLOTTED                    2175 non-null object
GRANTEE                    2175 non-null object
SALE_DATE                  2175 non-null object
EFFECTIVE_DATE_OF_LEASE    2175 non-null object
EXPIRATION_DATE            2175 non-null object
ACRES                      2108 non-null float64
COUNTY                     2128 non-null object
ST                         2133 non-null object
STATUS_COMMENTS            908 non-null object
EXPIRATION_YEAR            2076 non-null float64
TOWNSHIP                   1332 non-null object
RANGE_PLSS                 1331 non-null object
DESCRIPTION_I              1930 non-null object
DESCRIPTION_II             1868 non-null object
DESCRIPTION_III            844 non-null object
OFFICIAL_COUNTY            2175 non-null object
FIPS                       2175 non-null int64
YEARS_RENEWED         

In [77]:
internalShape.to_file('Output Lists/Tableau Final Shape/Internal Version/InternalShape.shp')

In [ ]:
#internalShape.to_file("Output Lists/INTERNAL_TableauShape.shp")